In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

## GET (SEMI-)ANNUAL AVERAGE RETURNS OVER TIME OF THE AVAILABLE FUNDS

In [ ]:
import numpy as np

In [ ]:
# Import CIKs from the dataframe of all cleaned filings
from utils import *
ciks = ', '.join(list(df_from_filings().cik.unique()))

In [ ]:
# Get all cik number available (intersection between our dataframe and the data on CRSP)
ciks_data = db.raw_sql(f'''
        select distinct comp_cik
        from crsp_q_mutualfunds.crsp_cik_map
        where comp_cik in ({ciks})
''')
cik_available = list(ciks_data.comp_cik)
cik_available = ', '.join([str(i) for i in cik_available])

In [ ]:
# With the mapping table of CRSP, get all companies' fund number
crsp_fundno_available = db.raw_sql(f'''
        select *
        from crsp_q_mutualfunds.crsp_cik_map
        where comp_cik in ({cik_available})
''')
crsp_fundno_available

In [ ]:
print(len(crsp_fundno_available['crsp_fundno'].unique()))

print(len(crsp_fundno_available['comp_cik'].unique()))

In [ ]:
# Convert the list of fund numbers to a list of string type for future SQL queries
list_float_fundnos = list(crsp_fundno_available.crsp_fundno.unique())
crsp_fundno_available_list_str = ', '.join([str(i) for i in list_float_fundnos])

In [ ]:
# Get all available returns for each fund
monthly_returns_all_funds = db.raw_sql(f'''
        select *
        from crsp_q_mutualfunds.monthly_returns
        where crsp_fundno in ({crsp_fundno_available_list_str})
        order by caldt desc
''')
monthly_returns_all_funds = pd.merge(left=monthly_returns_all_funds,right=crsp_fundno_available, how='inner', left_on='crsp_fundno', right_on='crsp_fundno')
monthly_returns_all_funds

In [ ]:
# Create a new dataframe with some statistic and the average return
monthly_returns_all_funds_aggregated = monthly_returns_all_funds[['caldt', 'mret', 'comp_cik', 'crsp_fundno']].groupby(by=['comp_cik','caldt']).agg(
    count=('crsp_fundno', 'count'),
    mret=('mret', 'mean'),
)

monthly_returns_all_funds_aggregated.sort_values(by=['comp_cik', 'caldt'], inplace=True)

# Create temporary column to compute later the cumulative return
monthly_returns_all_funds_aggregated['mret_add_1']=monthly_returns_all_funds_aggregated['mret']+1

# Compite the cumulative return of each CIK number
monthly_returns_all_funds_aggregated['cum_return']=monthly_returns_all_funds_aggregated.groupby(['comp_cik'])['mret_add_1'].cumprod()-1

# Display dataframe
display(monthly_returns_all_funds_aggregated)

In [ ]:
df_from_filings()

In [ ]:
df = monthly_returns_all_funds_aggregated.reset_index()
df[df['caldt'].dt.month ==6]

In [ ]:
monthly_returns_all_funds_aggregated.reset_index()['caldt'][0]